In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import json
from dateutil.parser import parse
import pandas as pd

In [2]:
url = 'https://outagemap.cobbemc.com/reports/02e190b3-64f2-48b0-81ba-295186233695'
chrome_driver_path = '/Users/xuanedx1/Desktop/URG/web-scaper/chromedriver'

In [3]:
def get_logs(url):

    desired_capabilities = DesiredCapabilities.CHROME
    desired_capabilities["goog:loggingPrefs"] = {"performance": "ALL"}

    # Create the webdriver object and pass the arguments
    options = webdriver.ChromeOptions()

    # Chrome will start in Headless mode
    options.add_argument('headless')

    # Ignores any certificate errors if there is any
    options.add_argument("--ignore-certificate-errors")

    # Startup the chrome webdriver with executable path and
    # pass the chrome options and desired capabilities as
    # parameters.
    driver = webdriver.Chrome(executable_path=chrome_driver_path,
                              chrome_options=options,
                              desired_capabilities=desired_capabilities)

    # Send a request to the website and let it load
    driver.get(url)

    # Sleeps for 5 seconds
    time.sleep(5)

    logs_raw = driver.get_log("performance")
    logs = [json.loads(lr["message"])["message"] for lr in logs_raw]
    
    return driver, logs


In [4]:
def summary_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "json" in log_["params"]["response"]["mimeType"]
        and "summary" in log_["params"]["response"]["url"]
    )

def report_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "json" in log_["params"]["response"]["mimeType"]
        and "report" in log_["params"]["response"]["url"]
    )

In [5]:
def get_layout_4_df(url):
    
    driver, logs = get_logs(url)
    
    summary_response = ""
    for log in filter(summary_filter, logs):
        request_id = log["params"]["requestId"]
        resp_url = log["params"]["response"]["url"]
        print(f"Caught {resp_url}")
        summary_response = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})['body']
        
    summary = json.loads(summary_response)
    timestamp = summary['summaryFileData']['date_generated']
    
    report_response = ""
    for log in filter(report_filter, logs):
        request_id = log["params"]["requestId"]
        resp_url = log["params"]["response"]["url"]
        print(f"Caught {resp_url}")
        report_response = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})['body']
        
    data = json.loads(report_response)

    layout_4_df = pd.DataFrame(columns = ['timestamp', 'zip_code', 'members_affected', 'members_served', 'number_outages'])
    for area in data['file_data']['areas']:

        row = {}
        row['timestamp'] = timestamp
        row['zip_code'] = area['name']
        row['members_affected'] = area['cust_a']['val']
        row['members_served'] = area['cust_s']
        row['number_outages'] = area['n_out']

        layout_4_df = pd.concat([pd.DataFrame([row]),layout_4_df.loc[:]]).reset_index(drop=True)
        
    return layout_4_df
    
    

In [6]:
get_layout_4_df(url)

/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_1887/3593499440.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path,
/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_1887/3593499440.py:18: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_driver_path,


Caught https://kubra.io/data/87ad9d80-9c38-40cd-9847-686b0ae2769d/public/summary-1/data.json


JSONDecodeError: Expecting value: line 1 column 1 (char 0)